## Setup

In [ ]:
!pip install einops ema_pytorch mat73 numpy scikit_learn torch tqdm wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IDL_Project

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1UTmW6iNsuZDOK5pC8i70YZC-1owajRkF/IDL_Project


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/IDL_Project')

## Load and Preprocess Data

In [ ]:
import numpy as np
import torch
import pandas as pd
import os
import random

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from DiffE.utils import zscore_norm, EEGDataset


In [ ]:
def load_eeg_data(root_dir, subject_id, data_type='thinking', eeg_channels=None, window_size=256, step_size=128):
    subject_id = str(subject_id).zfill(2)
    subject_dir = os.path.join(root_dir, subject_id)
    file_path = os.path.join(subject_dir, f"{data_type}.csv")

    df = pd.read_csv(file_path)
    if eeg_channels is None:
        non_eeg_cols = ['Time:256Hz', 'Epoch', 'Label', 'Stage', 'Flag']
        eeg_channels = [col for col in df.columns if col not in non_eeg_cols]

    eeg_data = df[eeg_channels].values  # shape: (n_samples, n_channels)
    labels = df['Label'].values
    unique_labels = np.unique(labels)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    labels = np.array([label_map[label] for label in labels])

    # Reshape data into epochs using sliding windows
    X = []
    Y = []

    for i in range(0, len(eeg_data) - window_size + 1, step_size):
        window_data = eeg_data[i:i+window_size, :].T  # Transpose to get (n_channels, n_timepoints)
        window_label = labels[i + window_size // 2]  # Use the label from the middle of the window
        X.append(window_data)
        Y.append(window_label)

    # Convert to torch tensors
    X = torch.tensor(np.array(X), dtype=torch.float32)
    Y = torch.tensor(np.array(Y), dtype=torch.long)

    # Apply z-score normalization
    X = zscore_norm(X)

    return X, Y

In [ ]:
def load_multiple_subjects(root_dir, subject_ids, data_type='thinking'):
    all_X = []
    all_Y = []

    for subject_id in subject_ids:
        try:
            X, Y = load_eeg_data(root_dir, subject_id, data_type)
            all_X.append(X)
            all_Y.append(Y)
            print(f"Loaded data from subject {subject_id}")
        except Exception as e:
            print(f"Error loading data from subject {subject_id}: {e}")

    # Combine data from all subjects
    X_combined = torch.cat(all_X, dim=0)
    Y_combined = torch.cat(all_Y, dim=0)

    return X_combined, Y_combined

In [ ]:
def split_subjects_and_data(root_dir, data_type='thinking', seen_ratio=0.9, train_ratio=0.9, val_ratio=0.15, seed=42):
    # Set random seed for reproducibility
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    # Get all subject IDs
    all_subject_ids = [str(i).zfill(2) for i in range(1, 22)]

    # Randomly shuffle subjects
    random.shuffle(all_subject_ids)

    # Split into seen and unseen
    num_seen = int(len(all_subject_ids) * seen_ratio)
    seen_subjects = all_subject_ids[:num_seen]
    unseen_subjects = all_subject_ids[num_seen:]

    print(f"Seen subjects: {seen_subjects}")
    print(f"Unseen subjects: {unseen_subjects}")

    # Load data for seen subjects
    seen_X, seen_Y = load_multiple_subjects(root_dir, seen_subjects, data_type)

    # Split seen subjects' data into train, val, test
    train_size = train_ratio
    val_size = val_ratio / (1 - train_ratio)  # Adjusted to be relative to remaining data after train split

    X_train, X_temp, Y_train, Y_temp = train_test_split(
        seen_X, seen_Y, test_size=(1-train_size), random_state=seed, stratify=seen_Y
    )

    X_val, X_seen_test, Y_val, Y_seen_test = train_test_split(
        X_temp, Y_temp, test_size=(1-val_size), random_state=seed, stratify=Y_temp
    )

    # Load data for unseen subjects (all for testing)
    X_unseen_test, Y_unseen_test = load_multiple_subjects(root_dir, unseen_subjects, data_type)

    # Create dataloaders
    batch_size_train = 32
    batch_size_test = 64

    train_dataset = EEGDataset(X_train, Y_train)
    val_dataset = EEGDataset(X_val, Y_val)
    seen_test_dataset = EEGDataset(X_seen_test, Y_seen_test)
    unseen_test_dataset = EEGDataset(X_unseen_test, Y_unseen_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size_test, shuffle=False)
    seen_test_loader = DataLoader(seen_test_dataset, batch_size=batch_size_test, shuffle=False)
    unseen_test_loader = DataLoader(unseen_test_dataset, batch_size=batch_size_test, shuffle=False)

    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Seen test samples: {len(seen_test_dataset)}")
    print(f"Unseen test samples: {len(unseen_test_dataset)}")

    return train_loader, val_loader, seen_test_loader, unseen_test_loader

In [ ]:
batch_size_train = 32
batch_size_test = 64
seed = 42
random.seed(seed)
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader, validation_loader, seen_test_loader, unseen_test_loader = split_subjects_and_data(
    root_dir="data_eeg",
    data_type='stimuli',
    seen_ratio=0.9,  # 18 seen, 3 unseen
    train_ratio=0.7,
    val_ratio=0.15,
    seed=42
)

Seen subjects: ['20', '06', '15', '05', '10', '14', '16', '19', '07', '13', '18', '11', '02', '12', '03', '17', '08', '09']
Unseen subjects: ['01', '04', '21']
Loaded data from subject 20
Loaded data from subject 06
Loaded data from subject 15
Loaded data from subject 05
Loaded data from subject 10
Loaded data from subject 14
Loaded data from subject 16
Loaded data from subject 19
Loaded data from subject 07
Loaded data from subject 13
Loaded data from subject 18
Loaded data from subject 11
Loaded data from subject 02
Loaded data from subject 12
Loaded data from subject 03
Loaded data from subject 17
Loaded data from subject 08
Loaded data from subject 09
Loaded data from subject 01
Loaded data from subject 04
Loaded data from subject 21
Train samples: 19811
Validation samples: 4245
Seen test samples: 4246
Unseen test samples: 4797


## Model

In [ ]:
import math
import numpy as np
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import reduce

In [ ]:
def get_padding(kernel_size, dilation=1):
    return int((kernel_size * dilation - dilation) / 2)


# Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)


class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


class WeightStandardizedConv1d(nn.Conv1d):
    """
    https://arxiv.org/abs/1903.10520
    weight standardization purportedly works synergistically with group normalization
    """

    def forward(self, x):
        eps = 1e-5 if x.dtype == torch.float32 else 1e-3

        weight = self.weight
        mean = reduce(weight, "o ... -> o 1 1", "mean")
        var = reduce(weight, "o ... -> o 1 1", partial(torch.var, unbiased=False))
        normalized_weight = (weight - mean) * (var + eps).rsqrt()

        return F.conv1d(
            x,
            normalized_weight,
            self.bias,
            self.stride,
            self.padding,
            self.dilation,
            self.groups,
        )


class ResidualConvBlock(nn.Module):
    def __init__(self, inc: int, outc: int, kernel_size: int, stride=1, gn=8):
        super().__init__()
        """
        standard ResNet style convolutional block
        """
        self.same_channels = inc == outc
        self.ks = kernel_size
        self.conv = nn.Sequential(
            WeightStandardizedConv1d(inc, outc, self.ks, stride, get_padding(self.ks)),
            nn.GroupNorm(gn, outc),
            nn.PReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.conv(x)
        if self.same_channels:
            out = (x + x1) / 2
        else:
            out = x1
        return out


class UnetDown(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, gn=8, factor=2):
        super(UnetDown, self).__init__()
        self.pool = nn.MaxPool1d(factor)
        self.layer = ResidualConvBlock(in_channels, out_channels, kernel_size, gn=gn)

    def forward(self, x):
        x = self.layer(x)
        x = self.pool(x)
        return x


class UnetUp(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, gn=8, factor=2):
        super(UnetUp, self).__init__()
        self.pool = nn.Upsample(scale_factor=factor, mode="nearest")
        self.layer = ResidualConvBlock(in_channels, out_channels, kernel_size, gn=gn)

    def forward(self, x):
        x = self.pool(x)
        x = self.layer(x)
        return x


class EmbedFC(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(EmbedFC, self).__init__()
        """
        generic one layer FC NN for embedding things
        """
        self.input_dim = input_dim
        layers = [
            nn.Linear(input_dim, emb_dim),
            nn.PReLU(),
            nn.Linear(emb_dim, emb_dim),
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        return self.model(x)


class ConditionalUNet(nn.Module):
    def __init__(self, in_channels, n_feat=256):
        super(ConditionalUNet, self).__init__()

        self.in_channels = in_channels
        self.n_feat = n_feat

        self.d1_out = n_feat * 1
        self.d2_out = n_feat * 2
        self.d3_out = n_feat * 3
        self.d4_out = n_feat * 4

        self.u1_out = n_feat
        self.u2_out = n_feat
        self.u3_out = n_feat
        self.u4_out = in_channels

        self.sin_emb = SinusoidalPosEmb(n_feat)
        # self.timeembed1 = EmbedFC(n_feat, self.u1_out)
        # self.timeembed2 = EmbedFC(n_feat, self.u2_out)
        # self.timeembed3 = EmbedFC(n_feat, self.u3_out)

        self.down1 = UnetDown(in_channels, self.d1_out, 1, gn=2, factor=2)
        self.down2 = UnetDown(self.d1_out, self.d2_out, 1, gn=2, factor=2)
        self.down3 = UnetDown(self.d2_out, self.d3_out, 1, gn=2, factor=2)

        self.up2 = UnetUp(self.d3_out, self.u2_out, 1, gn=2, factor=2)
        self.up3 = UnetUp(self.u2_out + self.d2_out, self.u3_out, 1, gn=2, factor=2)
        self.up4 = UnetUp(self.u3_out + self.d1_out, self.u4_out, 1, gn=2, factor=2)
        self.out = nn.Conv1d(self.u4_out + in_channels, in_channels, 1)

    def forward(self, x, t):
        down1 = self.down1(x)  # 2000 -> 1000
        down2 = self.down2(down1)  # 1000 -> 500
        down3 = self.down3(down2)  # 500 -> 250

        temb = self.sin_emb(t).view(-1, self.n_feat, 1)  # [b, n_feat, 1]

        up1 = self.up2(down3)  # 250 -> 500
        up2 = self.up3(torch.cat([up1 + temb, down2], 1))  # 500 -> 1000
        up3 = self.up4(torch.cat([up2 + temb, down1], 1))  # 1000 -> 2000
        out = self.out(torch.cat([up3, x], 1))  # 2000 -> 2000

        down = (down1, down2, down3)
        up = (up1, up2, up3)
        return out, down, up

class AttentionPool1d(nn.Module):
    def __init__(self, in_channels):
        super(AttentionPool1d, self).__init__()
        # Learnable query vector (shape: [in_channels])
        self.query = nn.Parameter(torch.randn(in_channels))

    def forward(self, x):
        """
        x: Tensor of shape (B, C, L) where B=batch size, C=channels, L=sequence length
        Returns a tensor of shape (B, C) by computing a weighted sum over L.
        """
        B, C, L = x.shape
        # Permute x to shape (B, L, C)
        x_perm = x.permute(0, 2, 1)  # (B, L, C)
        # Compute attention scores as dot-product between each time step and the query vector.
        # Resulting scores shape: (B, L)
        scores = torch.einsum('blc,c->bl', x_perm, self.query)
        # Softmax over the time dimension to get attention weights.
        weights = F.softmax(scores, dim=-1)  # (B, L)
        # Compute weighted sum over the time dimension.
        pooled = torch.sum(x * weights.unsqueeze(1), dim=2)  # (B, C)
        return pooled

class Encoder(nn.Module):
    def __init__(self, in_channels, dim=512):
        super(Encoder, self).__init__()

        self.in_channels = in_channels
        self.e1_out = dim
        self.e2_out = dim
        self.e3_out = dim

        self.down1 = UnetDown(in_channels, self.e1_out, 1, gn=2, factor=2)
        self.down2 = UnetDown(self.e1_out, self.e2_out, 1, gn=2, factor=2)
        self.down3 = UnetDown(self.e2_out, self.e3_out, 1, gn=2, factor=2)

        # self.avg_pooling = nn.AdaptiveAvgPool1d(output_size=1)
        self.att_pooling = AttentionPool1d(self.e3_out)
        # self.max_pooling = nn.AdaptiveMaxPool1d(output_size=1)
        self.act = nn.Tanh()

    def forward(self, x0):
        # Down sampling
        dn1 = self.down1(x0)  # 2048 -> 1024
        dn2 = self.down2(dn1)  # 1024 -> 512
        dn3 = self.down3(dn2)  # 512 -> 256
        # z = self.avg_pooling(dn3).view(-1, self.e3_out)  # [b, features]
        z = self.att_pooling(dn3)
        down = (dn1, dn2, dn3)
        out = (down, z)
        return out


class Decoder(nn.Module):
    def __init__(self, in_channels, n_feat=256, encoder_dim=512, n_classes=13):
        super(Decoder, self).__init__()

        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_classes = n_classes
        self.e1_out = encoder_dim
        self.e2_out = encoder_dim
        self.e3_out = encoder_dim
        self.d1_out = n_feat
        self.d2_out = n_feat * 2
        self.d3_out = n_feat * 3
        self.u1_out = n_feat
        self.u2_out = n_feat
        self.u3_out = n_feat
        self.u4_out = in_channels

        # self.sin_emb = SinusoidalPosEmb(n_feat)
        # self.timeembed1 = EmbedFC(n_feat, self.e3_out)
        # self.timeembed2 = EmbedFC(n_feat, self.u2_out)
        # self.timeembed3 = EmbedFC(n_feat, self.u3_out)
        # self.contextembed1 = EmbedFC(self.e3_out, self.e3_out)
        # self.contextembed2 = EmbedFC(self.e3_out, self.u2_out)
        # self.contextembed3 = EmbedFC(self.e3_out, self.u3_out)

        # Unet up sampling
        self.up1 = UnetUp(self.d3_out + self.e3_out, self.u2_out, 1, gn=2, factor=2)
        self.up2 = UnetUp(self.d2_out + self.u2_out, self.u3_out, 1, gn=2, factor=2)
        self.up3 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv1d(
                self.d1_out + self.u3_out + in_channels * 2, in_channels, 1, 1, 0
            ),
        )

        # self.out = nn.Conv1d(self.u4_out+in_channels, in_channels, 1)
        self.pool = nn.AvgPool1d(2)

    def forward(self, x0, encoder_out, diffusion_out):
        # Encoder output
        down, z = encoder_out
        dn1, dn2, dn3 = down

        # DDPM output
        x_hat, down_ddpm, up, t = diffusion_out
        dn11, dn22, dn33 = down_ddpm

        # embed context, time step
        # temb = self.sin_emb(t).view(-1, self.n_feat, 1) # [b, n_feat, 1]
        # temb1 = self.timeembed1(temb).view(-1, self.e3_out, 1) # [b, features]
        # temb2 = self.timeembed2(temb).view(-1, self.u2_out, 1) # [b, features]
        # temb3 = self.timeembed3(temb).view(-1, self.u3_out, 1) # [b, features]
        # ct2 = self.contextembed2(z).view(-1, self.u2_out, 1) # [b, n_feat, 1]
        # ct3 = self.contextembed3(z).view(-1, self.u3_out, 1) # [b, n_feat, 1]

        # Up sampling
        up1 = self.up1(torch.cat([dn3, dn33.detach()], 1))
        up2 = self.up2(torch.cat([up1, dn22.detach()], 1))
        out = self.up3(
            torch.cat([self.pool(x0), self.pool(x_hat.detach()), up2, dn11.detach()], 1)
        )
        return out


class DiffE(nn.Module):
    def __init__(self, encoder, decoder, fc):
        super(DiffE, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.fc = fc

    def forward(self, x0, ddpm_out):
        encoder_out = self.encoder(x0)
        z = encoder_out[1]
        decoder_out = self.decoder(x0, encoder_out, ddpm_out)
        fc_out = self.fc(encoder_out[1])
        return decoder_out, fc_out, z


class DecoderNoDiff(nn.Module):
    def __init__(self, in_channels, n_feat=256, encoder_dim=512, n_classes=13):
        super(DecoderNoDiff, self).__init__()

        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_classes = n_classes
        self.e1_out = encoder_dim
        self.e2_out = encoder_dim
        self.e3_out = encoder_dim
        self.u1_out = n_feat
        self.u2_out = n_feat
        self.u3_out = n_feat
        self.u4_out = n_feat

        self.sin_emb = SinusoidalPosEmb(n_feat)
        self.timeembed1 = EmbedFC(n_feat, self.e3_out)
        self.timeembed2 = EmbedFC(n_feat, self.u2_out)
        self.timeembed3 = EmbedFC(n_feat, self.u3_out)
        self.contextembed1 = EmbedFC(self.e3_out, self.e3_out)
        self.contextembed2 = EmbedFC(self.e3_out, self.u2_out)
        self.contextembed3 = EmbedFC(self.e3_out, self.u3_out)

        # Unet up sampling
        self.up2 = UnetUp(self.e3_out, self.u2_out, 1, gn=2, factor=2)
        self.up3 = UnetUp(self.e2_out + self.u2_out, self.u3_out, 1, gn=2, factor=2)
        # self.up4 = UnetUp(self.e1_out+self.u3_out, self.u4_out, 1, 1, gn=in_channels, factor=2, is_res=True)
        self.up4 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv1d(self.u3_out + self.e1_out + in_channels, in_channels, 1, 1, 0),
        )

        self.out = nn.Conv1d(self.u4_out, in_channels, 1)
        self.pool = nn.AvgPool1d(2)

    def forward(self, x0, x_hat, encoder_out, t):
        down, z = encoder_out
        dn1, dn2, dn3 = down
        tembd = self.sin_emb(t).view(-1, self.n_feat, 1)  # [b, n_feat, 1]
        tembd1 = self.timeembed1(self.sin_emb(t)).view(
            -1, self.e3_out, 1
        )  # [b, n_feat, 1]
        tembd2 = self.timeembed2(self.sin_emb(t)).view(
            -1, self.u2_out, 1
        )  # [b, n_feat, 1]
        tembd3 = self.timeembed3(self.sin_emb(t)).view(
            -1, self.u3_out, 1
        )  # [b, n_feat, 1]

        # Up sampling
        ddpm_loss = F.l1_loss(x0, x_hat, reduction="none")

        up2 = self.up2(dn3)  # 256 -> 512
        up3 = self.up3(torch.cat([up2, dn2], 1))  # 512 -> 1024
        out = self.up4(
            torch.cat([self.pool(x0), self.pool(x_hat), up3, dn1], 1)
        )  # 1024 -> 2048
        # out = self.out(torch.cat([out, x_hat], 1)) # 2048 -> 2048
        # out = self.out(out)
        return out


class LinearClassifier(nn.Module):
    def __init__(self, in_dim, latent_dim, emb_dim):
        super().__init__()
        self.linear_out = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=latent_dim),
            nn.GroupNorm(4, latent_dim),
            nn.PReLU(),
            nn.Linear(in_features=latent_dim, out_features=latent_dim),
            nn.GroupNorm(4, latent_dim),
            nn.PReLU(),
            nn.Linear(in_features=latent_dim, out_features=emb_dim),
        )

    def forward(self, x):
        x = self.linear_out(x)
        return x


def cosine_beta_schedule(timesteps, s=0.008):
    """
    cosine schedule
    as proposed in https://openreview.net/forum?id=-NEXDKk8gZ
    """
    steps = timesteps + 1
    t = torch.linspace(0, timesteps, steps, dtype=torch.float64) / timesteps
    alphas_cumprod = torch.cos((t + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0, 0.999)


def sigmoid_beta_schedule(timesteps, start=-3, end=3, tau=1, clamp_min=1e-5):
    """
    sigmoid schedule
    proposed in https://arxiv.org/abs/2212.11972 - Figure 8
    """
    steps = timesteps + 1
    t = torch.linspace(0, timesteps, steps, dtype=torch.float64) / timesteps
    v_start = torch.tensor(start / tau).sigmoid()
    v_end = torch.tensor(end / tau).sigmoid()
    alphas_cumprod = (-((t * (end - start) + start) / tau).sigmoid() + v_end) / (
        v_end - v_start
    )
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0, 0.999)


def ddpm_schedules(beta1, beta2, T):
    # assert beta1 < beta2 < 1.0, "beta1 and beta2 must be in (0, 1)"
    # beta_t = (beta2 - beta1) * torch.arange(0, T + 1, dtype=torch.float32) / T + beta1
    beta_t = cosine_beta_schedule(T, s=0.008).float()
    # beta_t = sigmoid_beta_schedule(T).float()

    alpha_t = 1 - beta_t

    log_alpha_t = torch.log(alpha_t)
    alphabar_t = torch.cumsum(log_alpha_t, dim=0).exp()

    sqrtab = torch.sqrt(alphabar_t)

    sqrtmab = torch.sqrt(1 - alphabar_t)

    return {
        "sqrtab": sqrtab,  # \sqrt{\bar{\alpha_t}}
        "sqrtmab": sqrtmab,  # \sqrt{1-\bar{\alpha_t}}
    }


class DDPM(nn.Module):
    def __init__(self, nn_model, betas, n_T, device):
        super(DDPM, self).__init__()
        self.nn_model = nn_model.to(device)

        for k, v in ddpm_schedules(betas[0], betas[1], n_T).items():
            self.register_buffer(k, v)

        self.n_T = n_T
        self.device = device

    def forward(self, x):
        _ts = torch.randint(1, self.n_T, (x.shape[0],)).to(
            self.device
        )  # t ~ Uniform(0, n_T)
        noise = torch.randn_like(x)  # eps ~ N(0, 1)
        x_t = self.sqrtab[_ts, None, None] * x + self.sqrtmab[_ts, None, None] * noise
        times = _ts / self.n_T
        output, down, up = self.nn_model(x_t, times)
        return output, down, up, noise, times

In [ ]:
num_classes = 16
channels = 14
n_T = 1000
ddpm_dim = 128
encoder_dim = 256
fc_dim = 512

In [ ]:
ddpm_model = ConditionalUNet(in_channels=channels, n_feat=ddpm_dim).to(device)
ddpm = DDPM(nn_model=ddpm_model, betas=(1e-6, 1e-2), n_T=n_T, device=device).to(device)
encoder = Encoder(in_channels=channels, dim=encoder_dim).to(device)
decoder = Decoder(in_channels=channels, n_feat=ddpm_dim, encoder_dim=encoder_dim).to(device)
fc = LinearClassifier(encoder_dim, fc_dim, emb_dim=num_classes).to(device)
diffe = DiffE(encoder, decoder, fc).to(device)

In [ ]:
print("ddpm size: ", sum(p.numel() for p in ddpm.parameters()))
print("encoder size: ", sum(p.numel() for p in encoder.parameters()))
print("decoder size: ", sum(p.numel() for p in decoder.parameters()))
print("fc size: ", sum(p.numel() for p in fc.parameters()))

ddpm size:  238278
encoder size:  137219
decoder size:  135832
fc size:  404498


## Train

In [ ]:
import wandb
wandb.login(key="fc35b6207578f4e85e34481be02780068223a3f6")

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm
from tqdm.auto import tqdm
from ema_pytorch import EMA
from tqdm import tqdm
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    precision_score,
    recall_score,
    top_k_accuracy_score,
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Criterion
criterion = nn.L1Loss(reduction='none')
criterion_class = nn.CrossEntropyLoss()

In [ ]:
lr_ddpm = 1e-4
lr_diffe = 1e-4
weight_decay = 0.01

optimizer1 = optim.AdamW(ddpm.parameters(), lr=lr_ddpm, weight_decay=weight_decay)
optimizer2 = optim.AdamW(diffe.parameters(), lr=lr_diffe, weight_decay=weight_decay)

In [ ]:
num_epochs = 100
scheduler1 = optim.lr_scheduler.CosineAnnealingLR(optimizer1, T_max=num_epochs, eta_min=1e-7)
scheduler2 = optim.lr_scheduler.CosineAnnealingLR(optimizer2, T_max=num_epochs, eta_min=1e-7)

In [ ]:
fc_ema = EMA(diffe.fc, beta=0.95, update_after_step=100, update_every=10)

In [ ]:
# Create directory for saving models if it doesn't exist
models_path = "saved_models"
os.makedirs(models_path, exist_ok=True)

In [ ]:
best_val_accuracy = 0.0

In [ ]:
class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features, labels):
        """
        features: [B, D] — z output from the encoder
        labels:   [B]    — integer type labels
        """
        device = features.device
        features = F.normalize(features, dim=1)              # Feature normalization
        batch_size = features.shape[0]

        # Construct positive sample mask
        labels = labels.contiguous().view(-1, 1)
        mask = torch.eq(labels, labels.T).float().to(device)  # [B, B], 1 if same class

        # Similarity matrix
        sim = torch.matmul(features, features.T) / self.temperature  # [B, B]

        # Exclude diagonal (self with self)
        logits_mask = torch.ones_like(mask) - torch.eye(batch_size).to(device)
        mask = mask * logits_mask
        sim = sim - 1e9 * (1 - logits_mask)  # Mask the diagonal with large negative value

        # Compute log-softmax
        exp_sim = torch.exp(sim)
        log_prob = sim - torch.log(exp_sim.sum(1, keepdim=True) + 1e-6)

        # Compute mean log-probability of positive samples for each instance
        mean_log_prob_pos = (mask * log_prob).sum(1) / (mask.sum(1) + 1e-6)

        # Mean negative log-likelihood
        loss = -mean_log_prob_pos.mean()
        return loss

In [ ]:
class ProjectionHead(nn.Module):
    def __init__(self, input_dim=256, proj_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, proj_dim),
            nn.ReLU(),
            nn.Linear(proj_dim, proj_dim)
        )

    def forward(self, z):
        return F.normalize(self.net(z), dim=1)

In [ ]:
def train_epoch(ddpm_model, diffe_model, ema_classifier, train_loader,
                optimizer1, optimizer2, scheduler1, scheduler2,
                criterion_recon, criterion_class, epoch, device, num_classes):
    """Runs a single training epoch."""
    ddpm_model.train()
    diffe_model.train()

    total_loss_supcon = 0.0
    total_loss_decoder = 0.0
    total_loss_c = 0.0
    total_loss = 0.0

    alpha = 1
    beta = min(1.0, epoch / 50) * 0.2
    gamma = min(1.0, epoch / 100) * 0.05
    supcon_loss = SupConLoss(temperature=0.07)
    proj_head = ProjectionHead(input_dim=256, proj_dim=128).to(device)

    progress_bar = tqdm(train_loader, desc=f"Training", leave=True)

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.type(torch.LongTensor).to(device)

        # --- Train DDPM ---
        optimizer1.zero_grad()
        x_hat, down, up, noise, t = ddpm_model(x)
        # Use the criterion_recon which has reduction='none'
        loss_ddpm_per_sample = criterion_recon(x_hat, x)  # No reduction param needed
        loss_ddpm = loss_ddpm_per_sample.mean()  # Average for DDPM backward pass
        loss_ddpm.backward()
        optimizer1.step()
        ddpm_out = x_hat, down, up, t

        # --- Train Diff-E ---
        optimizer2.zero_grad()
        decoder_out, fc_out, z = diffe_model(x, ddpm_out)
        #loss_gap = criterion(decoder_out, loss_ddpm.detach())
        loss_decoder = F.l1_loss(decoder_out, x_hat.detach())
        loss_c = criterion_class(fc_out, y)
        z_proj = proj_head(z)
        loss_supcon = supcon_loss(z_proj, y)
        loss = alpha * loss_c + beta * loss_supcon + gamma * loss_decoder
        loss.backward()
        optimizer2.step()

        # --- Update EMA ---
        ema_classifier.update()

        # --- Logging (Wandb) ---
        wandb.log({
            "train/batch_loss_decoder": loss_decoder.item(),
            "train/batch_loss_supcon": loss_supcon.item(),
            "train/batch_loss_c": loss_c.item(),
            "train/batch_loss_total": loss.item(),
            "train/learning_rate1": scheduler1.get_last_lr()[0],
            "train/learning_rate2": scheduler2.get_last_lr()[0]
        })

        # --- Accumulate Epoch Losses ---
        total_loss_decoder += loss_decoder.item()
        total_loss_supcon += loss_supcon.item()
        total_loss_c += loss_c.item()
        total_loss += loss.item()

    # --- Return Average Epoch Losses ---
    num_batches = len(train_loader)
    avg_losses = {
        "loss_decoder": total_loss_decoder / num_batches,
        "loss_supcon": total_loss_supcon / num_batches,
        "loss_c": total_loss_c / num_batches,
        "loss_total": total_loss / num_batches,
    }
    return avg_losses

In [ ]:
def evaluate(diffe_model, ddpm_model, ema_classifier, dataloader,
             criterion_recon, criterion_class, epoch, device, num_classes):
    """Evaluates the model on the provided dataloader, returning metrics and losses."""
    diffe_model.eval()
    ddpm_model.eval()
    ema_classifier.eval()

    all_labels = []
    all_preds = []
    total_loss_supcon = 0.0
    total_loss_decoder = 0.0
    total_loss_c = 0.0
    total_loss = 0.0

    alpha = 1
    beta = min(1.0, epoch / 50) * 0.2
    gamma = min(1.0, epoch / 100) * 0.05
    supcon_loss = SupConLoss(temperature=0.07)
    proj_head = ProjectionHead(input_dim=256, proj_dim=128).to(device)

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.type(torch.LongTensor).to(device)

            # DDPM forward pass
            x_hat, down, up, noise, t = ddpm_model(x)
            loss_ddpm_per_sample = criterion_recon(x_hat, x)
            loss_ddpm = loss_ddpm_per_sample.mean()
            ddpm_out = x_hat, down, up, t

            # DiffE forward pass
            decoder_out, fc_out, z = diffe_model(x, ddpm_out)
            #loss_gap = criterion_recon(decoder_out, loss_ddpm_per_sample).mean()

            # Classification loss
            y_hat = ema_classifier(diffe_model.encoder(x)[1])

            loss_decoder = F.l1_loss(decoder_out, x_hat.detach())
            loss_c = criterion_class(fc_out, y)
            z_proj = proj_head(z)
            loss_supcon = supcon_loss(z_proj, y)
            loss = alpha * loss_c + beta * loss_supcon + gamma * loss_decoder

            # For metrics calculation
            y_hat_softmax = F.softmax(y_hat, dim=1)
            all_labels.append(y.detach().cpu())
            all_preds.append(y_hat_softmax.detach().cpu())

            # Accumulate losses
            total_loss_decoder += loss_decoder.item()
            total_loss_supcon += loss_supcon.item()
            total_loss_c += loss_c.item()
            total_loss += loss.item()

    # Calculate average losses
    num_batches = len(dataloader)
    avg_losses = {
        "loss_decoder": total_loss_decoder / num_batches,
        "loss_supcon": total_loss_supcon / num_batches,
        "loss_c": total_loss_c / num_batches,
        "loss_total": total_loss / num_batches,
    }

    # Convert predictions and labels for metric calculation
    all_labels = torch.cat(all_labels, dim=0).numpy()
    all_preds = torch.cat(all_preds, dim=0).numpy()
    pred_classes = all_preds.argmax(axis=1)

    # Calculate metrics
    accuracy = (pred_classes == all_labels).mean()

    # For multi-class metrics, handle potential warnings
    try:
        f1 = f1_score(all_labels, pred_classes, average="macro", zero_division=0)
        recall = recall_score(all_labels, pred_classes, average="macro", zero_division=0)
        precision = precision_score(all_labels, pred_classes, average="macro", zero_division=0)
        auc = roc_auc_score(all_labels, all_preds, average="macro", multi_class="ovo")
    except ValueError as e:
        print(f"Warning in metric calculation: {e}")
        f1, recall, precision, auc = 0.0, 0.0, 0.0, 0.0

    metrics = {
        "accuracy": accuracy,
        "f1": f1,
        "recall": recall,
        "precision": precision,
        "auc": auc,
    }

    return metrics, avg_losses

In [ ]:
# Wandb configuration
config = {
    # Model parameters
    "model_type": "DiffE-EEG",
    "num_classes": num_classes,
    "channels": channels,
    "n_T": n_T,
    "ddpm_dim": ddpm_dim,
    "encoder_dim": encoder_dim,
    "fc_dim": fc_dim,

    # Training parameters
    "epochs": num_epochs,
    "train_batch_size": batch_size_train,
    "test_batch_size": batch_size_test,
    "seed": seed,

    # Optimizer parameters
    "optimizer": "AdamW",
    "learning_rate_ddpm": lr_ddpm,
    "learning_rate_diffe": lr_diffe,
    "weight_decay": weight_decay,

    # Scheduler parameters
    "scheduler": "CosineAnnealingLR",
    "scheduler_T_max": num_epochs,
    "scheduler_eta_min": 1e-7,

    # EMA parameters
    "ema_beta": 0.95,
    "ema_update_after_step": 100,
    "ema_update_every": 10
}

# Initialize wandb run
run = wandb.init(
    project="DiffE-EEG",
    config=config,
    name=f"diffE-FEIS-thinking",
    reinit=True  # For notebooks
)

In [ ]:
print("Starting training...")
epoch_progress = tqdm(range(num_epochs), desc="Epochs", position=0)

best_val_accuracy = 0.0

for epoch in epoch_progress:
    # --- Training ---
    train_losses = train_epoch(
        ddpm, diffe, fc_ema, train_loader,
        optimizer1, optimizer2, scheduler1, scheduler2,
        criterion, criterion_class, epoch, device, num_classes
    )

    # Step schedulers after full epoch
    scheduler1.step()
    scheduler2.step()

    # --- Evaluation ---
    epoch_progress.set_description(f"Evaluating Epoch {epoch+1}/{num_epochs}")
    val_metrics, val_losses = evaluate(diffe, ddpm, fc_ema, validation_loader, criterion, criterion_class, epoch, device, num_classes)

    # Update progress bar with metrics
    epoch_progress.set_postfix({
        'Train Loss': f"{train_losses['loss_total']:.4f}",
        'Val Loss': f"{val_losses['loss_total']:.4f}",
        'Val Acc': f"{val_metrics['accuracy']:.4f}",
        'Val F1': f"{val_metrics['f1']:.4f}"
    })

    # --- Wandb Logging (Epoch Level) ---
    wandb.log({
        "epoch": epoch + 1,
        # Training losses
        "train/epoch_loss_decoder": train_losses['loss_decoder'],
        "train/epoch_loss_supcon": train_losses['loss_supcon'],
        "train/epoch_loss_c": train_losses['loss_c'],
        "train/epoch_loss_total": train_losses['loss_total'],
        # Validation losses
        "val/loss_decoder": val_losses['loss_decoder'],
        "val/loss_supcon": val_losses['loss_supcon'],
        "val/loss_c": val_losses['loss_c'],
        "val/loss_total": val_losses['loss_total'],
        # Validation metrics
        "val/accuracy": val_metrics['accuracy'],
        "val/f1_score": val_metrics['f1'],
        "val/recall": val_metrics['recall'],
        "val/precision": val_metrics['precision'],
        "val/auc": val_metrics['auc']
    })

    # --- Checkpoint Saving ---
    current_accuracy = val_metrics['accuracy']
    if current_accuracy > best_val_accuracy:
        best_val_accuracy = current_accuracy
        best_model_epoch = epoch + 1
        epoch_progress.write(f"*** New best validation accuracy: {best_val_accuracy:.4f} at epoch {epoch+1} ***")
        wandb.log({"val/best_accuracy": best_val_accuracy})

        # Save the models
        torch.save(diffe.state_dict(), os.path.join(models_path, f"best_diffe_model_epoch{epoch+1}.pt"))
        torch.save(ddpm.state_dict(), os.path.join(models_path, f"best_ddpm_model_epoch{epoch+1}.pt"))
        epoch_progress.write(f"Models saved to {models_path}")

# --- End of Training ---
print("Training finished.")
print(f"Best validation accuracy achieved: {best_val_accuracy:.4f}")

Starting training...


Evaluating Epoch 1/100:   1%|          | 1/100 [00:16<26:28, 16.04s/it, Train Loss=2.7896, Val Loss=2.7753, Val Acc=0.0775, Val F1=0.0726]

*** New best validation accuracy: 0.0775 at epoch 1 ***
Models saved to saved_models



Evaluating Epoch 3/100:   3%|▎         | 3/100 [00:54<28:56, 17.91s/it, Train Loss=2.7808, Val Loss=2.8149, Val Acc=0.0784, Val F1=0.0743]

*** New best validation accuracy: 0.0784 at epoch 3 ***
Models saved to saved_models


...

*** New best validation accuracy: 0.4535 at epoch 60 ***


Evaluating Epoch 60/100:  60%|██████    | 60/100 [15:36<10:40, 16.02s/it, Train Loss=0.6898, Val Loss=4.3016, Val Acc=0.4535, Val F1=0.4534]

Models saved to saved_models



Evaluating Epoch 100/100: 100%|██████████| 100/100 [25:48<00:00, 15.48s/it, Train Loss=0.6317, Val Loss=4.5649, Val Acc=0.4497, Val F1=0.4495]

Training finished.
Best validation accuracy achieved: 0.4535


### Visualize EEG Signals

In [ ]:
import matplotlib.pyplot as plt

def visualize_signals_grid(original, masked, reconstructed, sample_idxs, channel=0):
    """
    Creates a grid figure showing for each sample:
      Column 1: Signal Before (original)
      Column 2: Signal Masked (noisy/masked)
      Column 3: Signal After (reconstructed)
    """
    num_samples = len(sample_idxs)
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 4 * num_samples), squeeze=False)
    for i, sample_idx in enumerate(sample_idxs):
        orig_np = original[sample_idx, channel].detach().cpu().numpy()
        masked_np = masked[sample_idx, channel].detach().cpu().numpy()
        recon_np = reconstructed[sample_idx, channel].detach().cpu().numpy()
        axes[i, 0].plot(orig_np, color='blue')
        axes[i, 0].set_title(f"Sample {sample_idx} - Signal Before")
        axes[i, 1].plot(masked_np, color='orange')
        axes[i, 1].set_title(f"Sample {sample_idx} - Signal Masked")
        axes[i, 2].plot(recon_np, color='green')
        axes[i, 2].set_title(f"Sample {sample_idx} - Signal After")
        for j in range(3):
            axes[i, j].set_xlabel("Time")
            axes[i, j].set_ylabel("Amplitude")
    fig.tight_layout()
    return fig

In [ ]:
best_diffe_path = os.path.join(models_path, f"best_diffe_model_epoch{best_model_epoch}.pt")
best_ddpm_path = os.path.join(models_path, f"best_ddpm_model_epoch{best_model_epoch}.pt")

if os.path.exists(best_diffe_path) and os.path.exists(best_ddpm_path):
    diffe.load_state_dict(torch.load(best_diffe_path))
    ddpm.load_state_dict(torch.load(best_ddpm_path))
    print(f"Loaded best models from epoch {best_model_epoch}")

Loaded best models from epoch 60


In [ ]:
with torch.no_grad():
    for x, _ in seen_test_loader:
        x = x.to(device)
        # Generate reconstruction
        x_hat, down, up, noise, t = ddpm(x)

        # Visualize multiple samples and channels
        samples_to_visualize = min(3, x.size(0))
        channels_to_visualize = min(3, x.size(1))

        # Visualize each channel
        for channel_idx in range(channels_to_visualize):
            sample_idxs = list(range(samples_to_visualize))
            fig = visualize_signals_grid(
                x[:samples_to_visualize],
                noise[:samples_to_visualize],  # Using the noise from the diffusion process
                x_hat[:samples_to_visualize],
                sample_idxs,
                channel=channel_idx
            )

            wandb.log({f"Final_EEG_Signals/Channel_{channel_idx}": wandb.Image(fig)})
            plt.close(fig)  # Close to free memory

        # We only need one batch for visualization
        break

# --- Finish Wandb Run ---
wandb.finish()

epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/batch_loss_c,████▇▇▇▅▅▅▃▃▄▄▃▃▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/batch_loss_decoder,█▅▄▇▅▇▃▄▄▃▄▄▃▄▂▃▃▅▃▃▄▂▃▄▂▄▂▂▄▄▃▂▁▃▂▃▂▂▂▄
train/batch_loss_supcon,▇█▆▆▄▂▁▅▂▄▃▄▃▅▅▃▄▅▄▅▅▃▅▄▅▃▆▄▄▄▅▄▆▄▅▃▅▃▁▃
train/batch_loss_total,█▇████▇▇█▅▃▂▃▁▁▂▁▁▂▁▁▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁
train/epoch_loss_c,█████▆▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch_loss_decoder,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch_loss_supcon,█▇▄▄▄▃▃▅▃▃▄▃▃▃▂▂▂▃▂▂▂▁▂▂▂▄▂▂▃▂▂▂▁▂▂▁▂▂▃▂
train/epoch_loss_total,█████▇▇▇▆▆▅▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate1,██████▇▇▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▁▁▁▁▁
train/learning_rate2,██████▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁


In [ ]:
# Evaluate on seen subjects
seen_metrics, seen_losses = evaluate(diffe, ddpm, fc_ema, seen_test_loader,
                                    criterion, criterion_class, epoch, device, num_classes)
print(f"Seen subjects test results: Accuracy={seen_metrics['accuracy']:.4f}, F1={seen_metrics['f1']:.4f}")

# Evaluate on unseen subjects
unseen_metrics, unseen_losses = evaluate(diffe, ddpm, fc_ema, unseen_test_loader,
                                        criterion, criterion_class, epoch, device, num_classes)
print(f"Unseen subjects test results: Accuracy={unseen_metrics['accuracy']:.4f}, F1={unseen_metrics['f1']:.4f}")

Seen subjects test results: Accuracy=0.4385, F1=0.4385
Unseen subjects test results: Accuracy=0.0638, F1=0.0628
